In [18]:
# imports
from plotly.offline import iplot
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import random
import networkx as nx

### import from google sheet

In [19]:
def import_data(sheet_name, sheet_id = "1gQ69WDBNpTIegl1pKKwLvN2KJfTRnird6_2P41_qwfo"):
    return pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={sheet_name}")

In [20]:
nodes = import_data("1157642109")
links = import_data("1435699897")
nodes.shape, links.shape

((35, 4), (45, 3))

### encoded faction/group categories as integers

In [21]:
nodes.group = pd.Categorical(nodes.group)
nodes['group_id'] = nodes.group.cat.codes
nodes.shape

(35, 5)

### Adding node sizes
Size of each node is based on mentions of said node character in the 'links' table. Uncomment code below to normalize node size

In [22]:
sizes = links["Target"] \
        .value_counts() \
        .sort_index() \
        .add(links["Source"] \
             .value_counts() \
             .sort_index(), 
             fill_value = 1)

nodes = nodes.merge(pd.DataFrame({"Source" : sizes.index,
                                  "Size" : sizes.values}), 
                    left_on = "name",
                    right_on = "Source",
                    how = "left") \
                    .drop("Source", axis = 1)
                                                      

nodes.Size = nodes.Size.fillna(nodes.Size.mean())

# a, b = 1, 2
# x, y = min(nodes.Size), max(nodes.Size)
# nodes.Size = (nodes.Size - x) / (y - x) * (b - a) + a

nodes.shape

(35, 6)

### set random (but consistent) group color
We can put specific desired colors later on

In [23]:
def random_color(i):
    random.seed(i)
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % ( r(), r(), r() )

In [24]:
nodes["group_color"] = nodes.group_id.apply(lambda x: random_color(x))

### create graph object and add nodes

In [25]:
dnd = nx.Graph() # define our base graph object

In [26]:
for idx, char in nodes.iterrows():
        dnd.add_node(char.name,
                     size = char.Size,
                     name = char.name,
                     color = random_color(char.group_id),
                     description = char.description,
                     weight = 1,
                     group = char.group) 

len(dnd.nodes()), len(nodes) # sanity check

(35, 35)

### Create node lookup object to easily get index from name
And to avoid slow dataframe query functions

In [27]:
node_lookup = dict(zip(nodes.name, nodes.index))

### define relation color group

In [28]:
rel_color_lookup = {"Friend" : "green", 
                    "Foe" : 'red', 
                    "Unknown" : 'blue'}
                    
links["rel_color"] = links.Relation.apply(lambda x: rel_color_lookup[x])

### Add edges (links) to graph object
Note that a relationship like kivani <-> cissa is actually the same as cissa <-> kivani and will get treated as just one relation.
This is because we are doing undirected nodes

In [29]:
# for each co-appearance between two characters, add an edge
for idx, link in links.iterrows():
    dnd.add_edge(node_lookup[link.Source], 
                 node_lookup[link.Target], 
                 weight = 1, 
                 color = link.rel_color)

In [30]:
dnd.number_of_edges(), links.shape # kivani <-> cissa == cissa <-> kivani, so shape is different

(24, (45, 4))

In [31]:
print("Nodes of graph: ", len(dnd.nodes()))
print("Edges of graps: ", len(dnd.edges()))

Nodes of graph:  35
Edges of graps:  24


In [36]:
nodes.head()

,name,group,description,session_introduced,group_id,Size,group_color
0,Cissa,Party,adventuring bard,1.0,8,16.0,#74BDC0
1,Kivani,Party,magic initiate,1.0,8,16.0,#74BDC0
2,Kuruntu,Party,bored security,1.0,8,6.0,#74BDC0
3,Timothy,Party,rapier killer,1.0,8,7.0,#74BDC0
4,Bellhop,Party,aspiring monster hunter,4.0,8,8.0,#74BDC0


In [33]:
def create_node_trace(node_cluster, name):
    return go.Scatter3d(x = node_cluster.Xn, 
                        y = node_cluster.Yn, 
                        z = node_cluster.Zn, 
                        mode = 'markers + text', 
                        name = name, 
                        marker = dict(symbol = 'circle', 
                                    size = node_cluster.Size, 
                                    color = node_cluster.group_color),
                        line = dict(color='rgb(125,125,125)', width=0.5),
                        text = node_cluster.name, 
                        visible = True,
                        hoverinfo = 'text',
                        showlegend = True,
                        customdata = np.stack([node_cluster.description, 
                                               node_cluster.group], 
                                               axis = 1),
                        hovertemplate = ('%{text}'+\
                                        '<br><i>%{customdata[0]}</i><br>'+\
                                        '<b>Affiliation</b>: %{customdata[1]}<br>'))

def create_edge_trace(edge_x, edge_y, edge_z, edge_cluster, name):
    return go.Scatter3d(x = edge_x, 
                        y = edge_y, 
                        z = edge_z,
                        name = name,
                        text = edge_cluster.Relation,
                        line = dict(width=1,
                                    color=links.rel_color),
                        visible = True,
                        hoverinfo= 'text',
                        mode='lines')


In [ ]:
def get_node_coordinates(graph_obj, layout):
    
    Xn, Yn, Zn = [], [], []
    for k in range(graph_obj.number_of_nodes()): 
        Xn += [ layout[k][0] ] 
        Yn += [ layout[k][1] ]
        Zn += [ layout[k][2] ]

    return Xn, Yn, Zn

def get_link_coordinates(graph_obj, nodes_df):
    

    edge_x, edge_y, edge_z = [], [], []
    for edge in graph_obj.edges():
        x0, y0, z0 = nodes_df.loc[edge[0]][["Xn", "Yn", "Zn"]].values
        x1, y1, z1 = nodes_df.loc[edge[1]][["Xn", "Yn", "Zn"]].values
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None) # I'll be perfectly honest. I have no idea why we append these None elements. But it works!
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_z.append(z0)
        edge_z.append(z1)
        edge_z.append(None)

    return edge_x, edge_y, edge_z

### Fit graph objects to chosen layout
We can try different layouts with different parameters

In [34]:
pos_ = nx.spring_layout(dnd, dim = 3, k = 0.8, iterations = 50) # get coordinates for nodes in spring layoyt 
# print(pos_)

### Placed nodes and add traces
(traces are effectively plots that are plotted on top of each other)

In [35]:
placed_nodes = pd.merge(nodes, pd.DataFrame(zip(*get_node_coordinates(dnd, pos_)), 
                                     columns = ["Xn", "Yn", "Zn"]),
                                     left_index = True,
                                     right_index = True)


node_trace = create_node_trace(placed_nodes,
                               name = "all")

edge_trace = create_edge_trace(*get_link_coordinates(dnd, placed_nodes), 
                               links, 
                               name = "edges")

axis = dict(showbackground = True, 
            showline = False, 
            zeroline = False, 
            showgrid = False, 
            showticklabels = False, 
            showspikes = False,
            title = '')

layout = go.Layout(
    title = "Campaign Cast Network",
    plot_bgcolor='#696969',
    width = 800,
    height = 600,
    showlegend = True,
    scene = dict(
        xaxis = dict(axis),
        yaxis = dict(axis),
        zaxis = dict(axis)))

fig = go.Figure(data = [edge_trace, node_trace], 
                layout = layout)

iplot(fig, filename = 'campaign')
